In [1]:
import numpy as np

from eitprocessing.datahandling.breath import Breath
from eitprocessing.datahandling.continuousdata import ContinuousData
from eitprocessing.datahandling.eitdata import EITData
from eitprocessing.datahandling.intervaldata import IntervalData
from eitprocessing.datahandling.loading import load_eit_data
from eitprocessing.datahandling.sequence import Sequence
from eitprocessing.features.breath_detection import BreathDetection

In [2]:
path = ...

In [3]:
sequence = load_eit_data(
    path,
    vendor="draeger",
)

In [4]:
sequence = sequence.t[55940:55970]

In [10]:
eit_data = sequence.eit_data["raw"]
continuous_data = sequence.continuous_data["global_impedance_(raw)"]

# bd_kwargs = self.breath_detection_kwargs.copy()
breath_detection = BreathDetection()
breaths = breath_detection.find_breaths(continuous_data)

middle_times = np.array(
    [np.argmax(eit_data.time == middle_time) for middle_time in [breath.middle_time for breath in breaths.values]],
)

_, rows, cols = eit_data.pixel_impedance.shape

pixel_inflations = np.empty((len(breaths), rows, cols), dtype=object)
for row in range(rows):
    for col in range(cols):
        end = []
        middle = []

        if not np.isnan(
            eit_data.pixel_impedance[:, row, col],
        ).any() and not np.all(
            eit_data.pixel_impedance[:, row, col] == 0.0,
        ):
            start = [
                np.argmin(
                    eit_data.pixel_impedance[
                        middle_times[i] : middle_times[i + 1],
                        row,
                        col,
                    ],
                )
                + middle_times[i]
                for i in range(len(middle_times) - 1)
            ]

            end = [start[i + 1] for i in range(len(start) - 1)]
            middle = [
                np.argmax(
                    eit_data.pixel_impedance[start[i] : start[i + 1], row, col],
                )
                + start[i]
                for i in range(len(start) - 1)
            ]

            time = eit_data.time

            inflations = [
                Breath(time[s], time[m], time[e])
                for s, m, e in zip(
                    start[:-1],
                    middle,
                    end,
                    strict=True,
                )
            ]
            # First and last inflation are not detected by definition (need two breaths to find one inflation)
            inflations = [None, *inflations, None]

        else:
            inflations = None
        pixel_inflations[:, row, col] = inflations

pixel_inflations_container = IntervalData(
    label="pixel inflations",
    name="Pixel in- and deflation timing as determined by PixelInflation",
    unit=None,
    category="breath",
    intervals=[(time[middle_times[i]], time[middle_times[i + 1]]) for i in range(len(middle_times) - 1)],
    values=pixel_inflations,
    parameters={},
    derived_from=[eit_data],
)

sequence.interval_data.add(pixel_inflations_container)